# tokenizing

In [12]:
tokens = [] # Global variable to hold a list of tokens

def tokenizer(line):
    "Return a list of the tokens on this line, handling spaces properly, and upper-casing."
    line = ''.join(tokenize(line)) # Remove whitespace
    return tokenize(line.upper())

def peek(): 
    """Return the first token in the global `tokens`, or None if we are at the end of the line.
    
    inspect the first element but dont remove.
    """
    return (tokens[0] if tokens else None)

def pop(constraint=None):
    """Remove and return the first token in `tokens`, or return None if token fails constraint.
    constraint can be None, a literal (e.g. pop('=')), or a predicate (e.g. pop(is_varname))."""
    top = peek()
    if constraint is None or (top == constraint) or (callable(constraint) and constraint(top)):
        return tokens.pop(0)
    
def remove_spaces(line): 
    "Remove white space from line, except space inside double quotes."
    return 

def lines(text): 
    "A list of the non-empty lines in a text."
    return [line for line in text.splitlines() if line]

# parsing
parsing maps source code into an internal representation, ie abstract syntax tree. in lisp, the source code mapped closely to an ast that the lisp interpreter could understand.

in basic, each line of code has the following format: a line number, a keyword, and expression (if necessary).

### what are keywords?
keywords include `let` (for assigning variables to expressions), `GOTO`, etc. keywords tell the basic interpreter what to do, or how to evaluate an expression.

In [2]:

def Grammar(): 
    return {
    'LET':    [variable, '=', expression],
    'READ':   [list_of(variable)],
    'DATA':   [list_of(number)],
    'PRINT':  [labels_and_expressions],
    'GOTO':   [linenumber],
    'IF':     [expression, relational, expression, 'THEN', linenumber],
    'FOR':    [varname, '=', expression, 'TO', expression, step],
    'NEXT':   [varname],
    'END':    [],
    'STOP':   [],
    'DEF':    [funcname, '(', varname, ')', '=', expression],
    'GOSUB':  [linenumber],
    'RETURN': [],
    'DIM':    [list_of(variable)], 
    'REM':    [anycharacters],
    'A':    []
    }

In [85]:

def number():        return (-1 if pop('-') else +1) * float(pop()) # Optional minus sign
def step():          return (expression() if pop('STEP') else 1)    # 1 is the default step

def linenumber():    
    """
    peek() returns the first element (but doesnt remove it from the list of tokens). 
    if peek() tells us that the first element is a number, then lets pop it. since it
    is a string, the parser needs to convert "20" into an integer 20.
    """
    return (int(pop()) if peek().isnumeric() else fail('missing line number'))

def relational():    return pop(is_relational) or fail('expected a relational operator')

def varname():       
    return pop(is_varname)    or fail('expected a variable name')

def funcname():      return pop(is_funcname)   or fail('expected a function name')
def anycharacters(): tokens.clear() # Ignore tokens in a REM statement
    
def is_stmt_type(x):  
    """
    first, we check that the token is a string. then we see if the string is a 
    reserved keyword, as defined by grammar.
    """
    return is_str(x) and x in grammar # LET, READ, ...

def is_funcname(x):   return is_str(x) and len(x) == 3 and x.isalpha()  # SIN, COS, FNA, FNB, ...
def is_varname(x):    
    return is_str(x) and len(x) in (1, 2) and x[0].isalpha() # A, A1, A2, B, ...
def is_label(x):      return is_str(x) and x.startswith('"') # "HELLO WORLD", ...
def is_relational(x): return is_str(x) and x in ('<', '=', '>', '<=', '<>', '>=')
def is_number(x):     return is_str(x) and x and x[0] in '.0123456789' # '3', '.14', ...

def is_str(x):        
    return isinstance(x, str)

def fail(message): 
    raise SyntaxError(message)


def variable(): 
    "Parse a possibly subscripted variable e.g. 'X3' or 'A(I)' or 'M(2*I, 3)'."
    V = varname()
    if pop('('):
        indexes = list_of(expression)()
        pop(')') or fail('expected ")" to close subscript')
        return Subscript(V, indexes) # E.g. 'A(I)' => Subscript('A', ['I'])
    else: 
        print("mogo1")
        return V                     # E.g. 'X3'
    
class list_of:
    "list_of(category) is a callable that parses a comma-separated list of <category>"
    def __init__(self, category): self.category = category
    def __call__(self):
        result = ([self.category()] if tokens else [])
        while pop(','):
            result.append(self.category())
        return result
    
def expression(prec=1): 
    "Parse an expression: a primary and any [op expression]* pairs with precedence(op) >= prec."
    exp = primary()                         # 'A' => 'A'
    while precedence(peek()) >= prec:
        op = pop()
        rhs = expression(precedence(op) + associativity(op))
        exp = Opcall(exp, op, rhs)          # 'A + B' => Opcall('A', '+', 'B')
    return exp

def labels_and_expressions():
    "Parse a sequence of label / comma / semicolon / expression (for PRINT statement)."
    result = []
    while tokens:
        item = pop(is_label) or pop(',') or pop(';') or expression()
        result.append(item)
    return result

def primary():
    "Parse a primary expression (no infix op except maybe within parens)."
    if is_number(peek()):                   # '1.23' => 1.23 
        return number()
    elif is_varname(peek()):                # X or A(I) or M(I+1, J)
        return variable()
    elif is_funcname(peek()):               # SIN(X) => Funcall('SIN', 'X')
        return Funcall(pop(), primary())
    elif pop('-'):                          # '-X' => Funcall('NEG', 'X')
        return Funcall('NEG', primary())
    elif pop('('):                          # '(X)' => 'X'
        exp = expression()
        pop(')') or fail('expected ")" to end expression')
        return exp
    else:
        return fail('unknown expression')
    
def precedence(op): 
    return (3 if op == '^' else 2 if op in ('*', '/', '%') else 1 if op in ('+', '-') else 0)

def associativity(op): 
    return (0 if op == '^' else 1)



In [86]:
from collections import namedtuple, defaultdict, deque

# for namedtuples, go https://pymotw.com/2/collections/namedtuple.html
Stmt      = namedtuple('Stmt',      'num, typ, args')     # '1 GOTO 9' => Stmt(1, 'GOTO', 9)
Subscript = namedtuple('Subscript', 'var, indexes')       # 'A(I)'     => Subscript('A', ['I'])
Funcall   = namedtuple('Funcall',   'f, x')               # 'SQR(X)'   => Funcall('SQR', 'X')
Opcall    = namedtuple('Opcall',    'x, op, y')           # 'X + 1'    => Opcall('X', '+', 1)
ForState  = namedtuple('ForState',  'continu, end, step') # Data for FOR loop 

class Function(namedtuple('_', 'parm, body')):
    "User-defined function; 'DEF FNC(X) = X ^ 3' => Function('X', Opcall('X', '^', 3))"
    def __call__(self, value):                           
        variables[self.parm] = value # Global assignment to the parameter
        return evalu(self.body)

In [92]:
def statement():
    """Parse a BASIC statement from `tokens`.
    
    this method does the heavy lifting.
    """
    # num is the line number
    num  = linenumber()  # pops line number from line
    
    # typ will be the keyword
    typ  = pop(is_stmt_type) or fail('unknown statement type')
    
    # args will be the expression
    args = []
    for p in grammar[typ]: # For each part of rule, call if callable or match if literal string
        if callable(p):  # cpython assumes all variables bound to functions are callable
            print(p.__name__)
            args.append(p())
        else:
            pop(p) or fail('expected ' + repr(p))
    return Stmt(num, typ, args)

In [93]:
grammar = Grammar()
tokens = ['20', 'LET', 'X', '=', 'X', '+', '1']

In [94]:
statement()

variable
mogo1
expression
mogo1


Stmt(num=20, typ='LET', args=['X', Opcall(x='X', op='+', y=1.0)])